In [ ]:
import matplotlib.pyplot as plt
%matplotlib widget
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.pyplot as plt
import chemiscope
from widget_code_input import WidgetCodeInput
from ipywidgets import Textarea
from iam_utils import *
import ase
from ase.io import read, write
import itertools

In [ ]:
#### AVOID folding of output cell 

In [ ]:
%%html

<style>
.output_wrapper, .output {
    height:auto !important;
    max-height:4000px;  /* your desired max-height here */
}
.output_scroll {
    box-shadow:none !important;
    webkit-box-shadow:none !important;
}
</style>

In [ ]:
data_dump = WidgetDataDumper(prefix="ex_02")
display(data_dump)

_Reference textbook / figure credits: Charles Kittel, *Introduction to solid-state physics", Chapter 1_

# Unit cells, fractional coordinates, lattices

A periodic structure is defined by a *lattice* and an a-periodic *repeat unit*. The lattice is a periodic set of points generated by all integer combinations of three *unit cell vectors* $\mathbf{a}_{1,2,3}$, i.e. 

$$
\mathbf{T} = u_1 \mathbf{a}_1 +  u_2 \mathbf{a}_2 +  u_3 \mathbf{a}_3, \quad u_{1,2,3} \in \mathbb{Z}
$$

The repeat unit, or *basis* is defined by the coordinates $\{x_i, y_i, z_i\}$ of a (usually small) number of atoms that sit in arbitrary positions within the unit cell. These are often given in *fractional coordinates* $\{s_{i1}, s_{i2}, s_{i3}\}$, such that the set of all atoms in the crystal is generated by 

$$
(u_1+s_{i1}) \mathbf{a}_1 +  (u_2+s_{i2}) \mathbf{a}_2 + (u_3+s_{i3})
$$

with the $u_{1,2,3}$ ranging over all positive and negative integers, and $i$ ranging over the number of atoms in the basis.

There are 14 types of lattices known as _Bravais lattices_ in three dimensions, that are distinguished by the symmetry group of the lattice points. If you need to refresh your fundamentals of crystallography, and don't remember what a _face centered cubic_ or _body centered cubic_ lattice is, the [wikipedia page](https://en.wikipedia.org/wiki/Bravais_lattice) is excellent. Note also that the crystal structure (lattice+basis) can have more complicated symmetries, forming the 230 [space groups](https://en.wikipedia.org/wiki/Space_group)

Now, consider the structure below. This is just a finite set of atoms, and coordinates are listed individually. You can click on the atoms in the viewer and see its coordinates by clicking on the info panel below the viewer.

In [ ]:
positions = np.array( [[x,y,z] for x,y,z in itertools.product([0, 5, 10, 15],[0, 5, 10, 15],[0, 5, 10, 15]) ])
ase_cube = ase.Atoms("Ga64", positions=positions)

properties = {}
properties["index"] = {
                "target": "atom",
                "values": list(range(1, len(ase_cube)+1)),
            }

properties["coordinates"] = {
                "target": "atom",
                "values": positions,
            }

cs = chemiscope.show([ase_cube], properties=properties, mode="structure",                      
                     environments=chemiscope.all_atomic_environments([ase_cube], cutoff=40),
                     settings={"structure":[{"spaceFilling": False, "environments": {"cutoff": 40}}]}
                    )
display(cs)


<span style="color:blue"> **01** Write a function that returns the lattice vectors and a basis that generates the periodic structure that continues to infinity the motif above. Try also to shift rigidly the basis atoms. Does the resulting crystal change in a significant way? </span>

_NB: the validation code only tests for the primitive cell - you can come up with correct structures that will be marked as incorrect. Trust your own judgment (and look carefully at the visualization)_

In [ ]:
ex01_wci = WidgetCodeInput(
        function_name="sc_lattice_basis", 
        function_parameters="",
        docstring="""
Returns the lattice vectors and the basis (in fractional coordinates) that generates a simple cubic structure.

:return: Three lattice vectors and a list of basis coordinates
""",
            function_body="""
# Write your solution, then click on the button below to update the plotter 
# and check against the reference value

a1 = []
a2 = []
a3 = []
basis = [[]]

return a1, a2, a3, basis
"""
        )

data_dump.register_field("ex01-function", ex01_wci, "function_body")

In [ ]:
def ex01_updater():
    a1, a2, a3, basis = ex01_wci.get_function_object()()
    positions = np.asarray(basis)
    h = np.asarray([a1,a2,a3])
    
    structure = ase.Atoms('Ga'+str(len(basis)), positions=positions@h.T, cell=[a1, a2, a3])
    display(chemiscope.show(frames = [structure], mode="structure", 
                            settings={"structure":[{"unitCell":True,"supercell":{"0":3,"1":3,"2":3}}]}
                           ))

def match_lattice(a, b):
    a1, a2, a3, basis = b;
    
    return np.allclose([a1,a2,a3], np.eye(3)*5) and np.asarray(basis).shape==(1,3)
    
"""ref_values = {
        (): ase.Atoms("CNH6", positions=[[ 1.,   -0.,   -0.01],
 [ 2.52, -0.01,  0.  ],
 [ 0.6 ,  1.02, -0.  ],
 [ 0.59, -0.52,  0.88],
 [ 0.6 , -0.51, -0.9 ],
 [ 2.92,  0.5 ,  0.89],
 [ 2.93,  0.51, -0.88],
 [ 2.92, -1.03, -0.  ]]) 
       }, ref_match = match_structure, """    
ex01_wcc = WidgetCodeCheck(ex01_wci, ref_values = { (): ([5,0,0],[0,5,0],[0,0,5],[[0,0,0]]) },
                           ref_match = match_lattice,
                           demo=WidgetUpdater(updater=ex01_updater))    
display(ex01_wcc)

In [ ]:
ex01_txt = Textarea("enter any additional comment", layout=Layout(width="100%"))
data_dump.register_field("ex01-answer", ex01_txt, "value")
display(ex01_txt)

## Primitive cell, supercells, conventional cells

It is always possible to give different descriptions to the same crystal structure. For instance, for a given cell and basis one can always specify a _supercell_ i.e. a multiple of repeat units of the original cell, with a correspondingly larger basis. This basis contains "hidden" symmetries, meaning that it would be possible to describe the same structure with a smaller unit cell. The smallest possible cell is called a _primitive_ (or minimal) cell. 

For instance, the figure below shows three different choices of unit cell for a face-centered cubic lattice. All cells describe the same structure! The first structure is the primitive cell, and the third the conventional cells that reveals more clearly the origin of the name of this lattice.

<img src="figures/fcc-cells.png" width="800" height="250" />

The following visualizer shows a collection of 9 crystal structures, containing either one or two chemical species. Look at them, and try to understand what type of lattice they belong to. You can visualize the environment of each atom within an adjusable cutoff, which may help you appreciate the 3D nature of the structure.

In [ ]:
ase_crystals = read('data/crystals.xyz',":")

properties = {}
properties["lattice vectors a1"] = {
                "target": "structure",
                "values": np.asarray([str(list(a.cell[0])) for a in ase_crystals]),
            }
properties["lattice vectors a2"] = {
                "target": "structure",
                "values": np.asarray([str(list(a.cell[1])) for a in ase_crystals]),
            }
properties["lattice vectors a3"] = {
                "target": "structure",
                "values": np.asarray([str(list(a.cell[2])) for a in ase_crystals]),
            }

properties["basis"] = {
                "target": "atom",
                "values": np.vstack([a.positions for a in ase_crystals]),
            }


cs = chemiscope.show(ase_crystals, properties=properties, mode="structure",                      
                     environments=chemiscope.all_atomic_environments(ase_crystals),
                     settings={"structure":[{"bonds":True, "unitCell":True,"supercell":{"0":3,"1":3,"2":3},
                                            "environments": {"cutoff": 6}}]}                    
                    )

def update_co(change):
    cs.settings={"structure": [{"environments": {"cutoff": pb.value['co']}}]}
pb = WidgetParbox(onchange=update_co, co=(6.,1,12,0.1, r"environment cutoff / Å"))
display(VBox([pb,cs]))


<span style="color:blue"> **02** Each of the structures above are either face-centered (fcc), body-centered (bcc) or simple cubic (sc). Write down in the box below what is the Bravais lattice for each structure, and the size of the basis used to describe the structure in each frame. </span>

_NB: pay attention: (1) you are asked about the symmetry of the **lattice**: if you just look at the cell, you may be misled; (2) the number of atoms included in the basis depend on the choice of cell._

In [ ]:
ex02_txt = Textarea("structure 1:  lattice: XXX, basis size: YYY\n ....", layout=Layout(width="100%"))
data_dump.register_field("ex02-answer", ex02_txt, "value")
display(ex02_txt)